In [1]:
import os, sys
from google.colab import drive
import shutil

drive.flush_and_unmount()
if os.path.isdir('/content/mnt')==False:
  drive.mount('/content/mnt')
  nb_path = '/content/notebooks'
  os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
  sys.path.insert(0, nb_path)  # or append(nb_path)

# Install only once. Tomorrow, you can skip this.
# !pip install --target=$nb_path spacy

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/mnt


In [93]:
import requests 
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from googlesearch import search as gs
import re
import pandas
import time

In [ ]:
# !pip install --target=$nb_path fake_useragent

In [ ]:
ua = UserAgent()
headers = {
    'User-agent' : ua.random
}
year = 2000
url = f'http://billboardtop100of.com/{year}-2/'
r   = requests.get(url,headers = headers)

k = 0
id = 4748
songs_dict = []

for year in range(2000,2021):

  fail = False;

  print('--------------------------------------------')
  print(f'Scraping year {year}')
  print('--------------------------------------------')
  url = f'http://billboardtop100of.com/{year}-2/'

  r   = requests.get(url,headers = headers)
  soup = bs(r.text,'lxml')
  try : # page format with tables tr
    print('Method 1')
    print('--------------------------------------------')
    table = soup.find('table',{'class': ' alignleft'})
    total_song  = table.find_all('tr')

    # from year 2000

    for i in total_song:
      songs_dict.append({
      'Number' : k + 1,
      'Artist' : i.find_all('td')[1].text,
      'Song'   : i.find_all('td')[2].text,
      'Year'   : soup.title.text.split(' ')[0]
                
      })
      k+=1
      print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
  except:
     #page format changed
    print('Method 2 ')
    print('--------------------------------------------')
    try:

      songs = soup.find('small')
      pattern = '((\d)*. )'
      for i in songs.text.split('\n'):
        full_line = i.replace('\xa0','')
        p = re.match(pattern,full_line)
        line = full_line.replace(p[0],'')
        line = line.split('–')
        try:
          song =  line[1]
        except:
          song = 'NaN'
        songs_dict.append({
          'Number' : k + 1,
          'Artist' : line[0],
          'Song'   : song,
          'Year'   : soup.title.text.split(' ')[0]
                
          })
        k+=1
        print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])


    except:
        try:
          #3rd try function.
          print('Method 3 ')
          print('--------------------------------------------')
          songs  = soup.find('table',{'class':'sortable alternate songtable'})
          tables = songs.find_all('tr')
          for i in tables:
            songs_dict.append({
            'Number' : k + 1,
            'Artist' : i.find_all('td')[1].text.replace('\n',''),
            'Song'   : i.find_all('td')[2].text.replace('\nLYRICS','').replace('\n',''),
            'Year'   : soup.title.text.split(' ')[0]
                
            
            })
            k+=1
            print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])

        except:
          
          try:
            #4rd try function.
            print('Method 4 ')
            print('--------------------------------------------')
            songs  = soup.find_all('figure',{'class': 'wp-block-image'})
            for i in songs:
              song = i.find_next('p')
              artist = song.find_next('p')
              songs_dict.append({
              'Number' : k + 1,
              'Artist' : artist.text.replace('\n',''),
              'Song'   : song.text.replace('\n',''),
              'Year'   : soup.title.text.split(' ')[0]
                
              
              })
              k+=1
              print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
          finally:
            try:

              # 5th try function.
              print('Method 5 ')
              print('--------------------------------------------')
              songs  = soup.find_all('div',{'class': 'wp-block-media-text__content'})

              for i in songs:
                try :

                  song = i.find_all('strong')[1]
                  artist = i.find('a').text
                except:
                  song = i.find('p')
                  artist = song.find_next('p').text

                songs_dict.append({
                'Number' : k + 1,
                'Artist' : artist,
                'Song'   : song.text,
                'Year'   : soup.title.text.split(' ')[0]
                
                })
                k+=1
                print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
            finally:

                fail = True
                print('UGH THAT\'S LIKE 5 METHOD\'S')

  if fail ==True :
    print('Final Approach ')
    print('--------------------------------------------')
    try:
      songs  = soup.find_all('article',{'class': 'ye-chart-item'})

      for i in songs:
      
        song = i.find(class_ = 'ye-chart-item__title')
        artist = i.find(class_ = 'ye-chart-item__artist')


        songs_dict.append({
        'Number' : k + 1,
        'Artist' : artist.text,
        'Song'   : song.text,
        'Year'   : soup.title.text.split(' ')[0]
        
        })
        k+=1
        print(songs_dict[k-1]['Number'],songs_dict[k-1]['Artist'],songs_dict[k-1]['Song'])
    except:
      print('This is so long')

print('-----------------------------------------------------------------')
print(' End of Program.')
print(f' Amount of data scraped : {len(songs_dict)}')

In [ ]:
df = pandas.DataFrame(songs_dict)
df.to_csv(os.path.join('/content/mnt/MyDrive/Colab Notebooks','top100BillboardSongs.csv'))

In [ ]:

all_links   = []
ytlinks = []
count = 0;
for i in df:
  # i = 10
  Song = i['Song']
  Artist = i['Artist']

  Search = f'{Artist} - {Song} lyrics'
  print(f'{count}. Searched : {Search}')

  LINKS = list(gs(Search, tld='com',  num=10, start=0,stop=15,pause=2.0))
  res   = {
      'Results': LINKS,
      'Artist' : Artist,
      'Song'   : Song

  }
  all_links.append(res)
  count+=1

results = pandas.DataFrame(all_links)
results.to_csv(os.path.join('/content/mnt/MyDrive/Colab Notebooks','MusicLyricsLinks.csv'))

In [ ]:
results = pandas.DataFrame(all_links)
results.to_csv(os.path.join('/content/mnt/MyDrive/Colab Notebooks','MusicLyricsLinks.csv'))

In [ ]:
results.head()

,Results,Artist,Song
0,"[https://genius.com/Faith-hill-breathe-lyrics,...",Faith Hill,Breathe
1,"[https://www.youtube.com/watch?v=guG1JbozB2E, ...",Santana feat. Rob Thomas,Smooth
2,[https://genius.com/The-product-g-b-maria-mari...,Santana feat. The Product GandB,Maria Maria
3,"[https://genius.com/Joe-i-wanna-know-lyrics, h...",Joe,I Wanna Know
4,"[https://www.youtube.com/watch?v=rZoD8JEFjAE, ...",Vertical Horizon,Everything You Want


In [ ]:

pattern_01 = r'\b(.lyrics.com)\b'
pattern_02 = r'\b(.youtube.)\b'
l = 1
m = 1
lyrics_links = []
ytlinks      = []
missing      = []
# youtube      = False
# found_ly     = False
# found_yt     = False
for i in range(0,len(results)):

  found_ly     = False
  found_yt     = False
  count = 0
  #print('--------------------------------------')
  for j in range(0,len(results[f'Results'][i])):
    k = results['Results'][i][j]
    rmatch  = re.search(pattern_01,k)
    youtube = re.search(pattern_02,k)
    if rmatch :
      found_ly = True
      lyrics_links.append(k)
      #print(l,'. ',k)
      l+=1
    
    if youtube and count<1:
      found_yt = True
      count = 2
      ytlinks.append(k)
      #print(m,'. ',k)
      m+=1
  if found_ly == False or found_yt == False:



    if found_ly == False:
      MISSING  = 'Lyrics Link'
    elif found_yt == False:
      MISSING  = 'Youtube Link'
    else:
      MISSING  = 'Both'
    MISS = {
        'Songs ' : results['Song'][i],
        'Artist' : results['Artist'][i],
        'Missing': MISSING

    }
    missing.append(MISS)

In [ ]:
MISSING_DATA = pandas.DataFrame(missing)

In [ ]:
MISSING_DATA[:87]
#export links : 
pandas.DataFrame(MISSING_DATA).to_csv('Missing_Links.csv',index = False)

***Searching for other lyrics formation as Azlyrics have AntiScraping in place.***

In [17]:
df = pandas.read_csv('/content/mnt/MyDrive/Colab Notebooks/MusicLyricsLinks.csv',index_col=False)

In [21]:
results = df
results.head()

,Unnamed: 0,Results,Artist,Song
0,0,['https://genius.com/Faith-hill-breathe-lyrics...,Faith Hill,Breathe
1,1,['https://www.youtube.com/watch?v=guG1JbozB2E'...,Santana feat. Rob Thomas,Smooth
2,2,['https://genius.com/The-product-g-b-maria-mar...,Santana feat. The Product GandB,Maria Maria
3,3,"['https://genius.com/Joe-i-wanna-know-lyrics',...",Joe,I Wanna Know
4,4,['https://www.youtube.com/watch?v=rZoD8JEFjAE'...,Vertical Horizon,Everything You Want


THIS JUST LOADING DATA FROM THE CSV FILE AND NOT FROM WEBSCRAPING


---



In [31]:
results['Results'][0]
# lyricsfreak
# metrolyrics
# lyricsmode
# songfacts
# songlyrics

"['https://genius.com/Faith-hill-breathe-lyrics', 'https://www.youtube.com/watch?v=yCmsZUN4r_s', 'https://www.youtube.com/watch?v=fPx-HQzBQ2M', 'https://www.youtube.com/watch?v=bx6DUQ7uqEw', 'https://www.azlyrics.com/lyrics/faithhill/breathe.html', 'https://www.metrolyrics.com/breathe-lyrics-faith-hill.html', 'http://www.songlyrics.com/faith-hill/breathe-lyrics/', 'https://www.lyrics.com/lyric/17683153/Faith+Hill/Breathe+%5BRemastered+Album+Version%5D', 'https://www.lyrics.com/lyric/3190469/Faith+Hill/Breathe', 'https://www.songfacts.com/lyrics/faith-hill/breathe', 'https://www.lyricsfreak.com/f/faith+hill/breathe_20052493.html', 'https://songmeanings.com/songs/view/7289/', 'https://en.wikipedia.org/wiki/Breathe_(Faith_Hill_song)', 'https://www.lyricsmode.com/lyrics/f/faith_hill/breathe_lyrics.html', 'https://www.songfacts.com/facts/faith-hill/breathe']"

In [37]:
import ast

In [ ]:
#Changing pandas array string to array

df['Results'] = df['Results'].apply(ast.literal_eval)



for i in results['Results']:
  print(i)

In [84]:
# lyricsfreak
# metrolyrics
# lyricsmode

# songfacts
# songlyrics

pattern_01 = r'\b(.metrolyrics.com)\b'
pattern_02 = r'\b(.youtube.)\b'
l = 1
m = 1
lyrics_links = []
ytlinks      = []
missing      = []
# youtube      = False
# found_ly     = False
# found_yt     = False
for i in range(0,len(results)):

  found_ly     = False
  # found_yt     = False
  count = 0
  #print('--------------------------------------')
  # for i in results['Results']:
  for j in range(0,len(results['Results'][i])):
    

    k = results['Results'][i][j]
    rmatch  = re.search(pattern_01,k)
    youtube = re.search(pattern_02,k)
    if rmatch :
      found_ly = True
      data = dict( {
          'Link'   : k,
          'Artist' : results['Artist'][i],
          'Song'   : results['Song'][i]
      })
      lyrics_links.append(data)
      #print(l,'. ',k)
      
      l+=1
      break

In [57]:
lyricsFreak 
LyricsMetro 
LyricsMode
LyricsSong
LyricsSongsL 
print('FREAK : ',len(lyricsFreak),'\nMETRO :',len(LyricsMetro),'\nMODE :',len(LyricsMode),'\nSONG :',len(LyricsSong),'\nSONGL :',len(LyricsSongsL))

FREAK :  991 
METRO : 1725 
MODE : 983 
SONG : 750 
SONGL : 1254


In [88]:
count = 1
for i in lyrics_links:
  print(count , i['Song'])
  count+=1

1 Breathe
2 I Wanna Know
3 Say My Name
4 I Knew I Loved You
5 Amazed
6 Bent
7 He Wasn’t Man Enough
8 Higher
9 Try Again
10 Jumpin Jumpin’
11 Thong Song
12 Kryptonite
13 There You Go
14 Music
15 Doesn’t Really Matter
16 What A Girl Wants
17 Back At One
18 Bye Bye Bye
19 I Need To Know
20 Get It On Tonite
21 Incomplete
22 It’s Gonna Be Me
23 That’s The Way It Is
24 (Hot S**T) Country Grammar
25 Bring It All To Me
26 Show Me The Meaning Of Being Lonely
27 Back Here
28 It Feels So Good
29 Absolutely (Story Of A Girl)
30 With Arms Wide Open
31 Be With You
32 Come On Over Baby (All I Want Is You)
33 No More
34 All The Small Things
35 The Way You Love Me
36 I Turn To You
37 Never Let You Go
38 I Need You
39 Let’s Get Married
40 My Love Is Your Love
41 Then The Morning Comes
42 Blue (Da Ba Dee)
43 The Real Slim Shady
44 Most Girls
45 Wifey
46 Wonderful
47 Oops!… I Did It Again
48 I Wanna Love You Forever
49 Give Me Just One Night (Una Noche )
50 Take A Picture
51 Otherside
52 Big Pimpin’
53 Pu

In [90]:
lyrics_links[1690]

{'Artist': 'Chris Brown Featuring Drake',
 'Link': 'https://www.metrolyrics.com/no-guidance-lyrics-chris-brown.html',
 'Song': '36\xa0\xa0No Guidance'}

Great news is that Metro lyrics site is scrapable by request. 



here's copy of robots.txt

User-agent: *
Allow: /api/v1/sitemapxml/*
Disallow: /tools/
Disallow: /api/
Disallow: /8264/
Disallow: /search.html
Disallow: /*-pictures.html$
Allow: /*-lyrics-*-pictures.html$

User-agent: ccbot
Disallow: /

User-Agent: msnbot
Crawl-delay: 2

User-Agent: bingbot
Crawl-delay: 2

User-Agent: Yandex
Crawl-delay: 2

In [ ]:
ALL_LYRICS = []


In [110]:

headers = {
    'authority': 'https://www.azlyrics.com',
    'dnt': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}

s = requests.Session()



count= 0
for k in lyrics_links[78:]:
  lyrics     = []
  # link = 'https://www.metrolyrics.com/breathe-lyrics-faith-hill.html'
  link = k['Link']
  site = s.get(link , headers = headers)
  soup = bs(site.text,'lxml')
  try:

    lyricsClass = soup.find('div',{'class':'lyrics-body'})

    for i in lyricsClass.find_all('p',{'class':'verse'}):
      lyrics.append(i.text)

    WORDS = {
        'Lyrics' :lyrics,
        'Artist' :k['Artist'],
        'Song'   :k['Song']
    }
    print(count,soup.title)
    ALL_LYRICS.append(WORDS)
    print('--------------------------------------------------')
    print(WORDS['Lyrics'])
    print('--------------------------------------------------')
    time.sleep(3.5)
    count+=1
  except:
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    print(soup.title, ' isn\'t Available')
    print('$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$')
    time.sleep(4)



Streaming output truncated to the last 5000 lines.
['Ran into an old friend yesterday\nCaught me by surprise when he called my name\nHe was a familiar face\nFrom a chapter in my past', "Talked for awhile, asked him how he's been\nSaid that he was seeing somebody else\nTold me this was gonna last\nShowing me her photograph", 'And all the feelings that I thought were gone\nCame rushing back to me at once\nTried to smile and hide the way I felt\nBut I was thinking to myself', "(Truth is)\nI never got over you\n(Truth is)\nWish I was standing in her shoes\n(Truth is)\nAnd when it's all said and done\nGuess I'm still in love with you", "(Truth is)\nI never should have let you go\n(Truth is)\nAnd it's killing me 'cause, now I know\n(Truth is)\nWhen it's all said and done\nGuess I'm still in love with you", 'We reminisce on the way things used to be\nShared a couple laughs and some memories\nTalked about the things that changed\nSome for good and some for bad', "Then he said goodbye and he pa

In [112]:
temp_save[:2]

[{'Artist': 'Faith Hill',
  'Lyrics': ["I can feel the magic floating in the air\nBeing with you gets me that way\nI watch the sunlight dance across your face\nAnd I've never been this swept away\nAll my thoughts just seem to settle on the breeze\nWhen I'm lying wrapped up in your arms\nThe whole world just fades away\nThe only thing I hear is the beating of your heart",
   "Cause I can feel you breathe\nIt's washing over me\nAnd suddenly I'm melting into you\nThere's nothing left to prove\nBaby all we need is just to be\nCaught up in the touch\nSlow and steady rush\nBaby isn't that the way that love's supposed to be?\nI can feel you breathe\nJust breathe",
   "In a way I know my heart is waking up\nAs all the walls come tumbling down\nCloser than I've ever felt before\nAnd I know, and you know\nThere's no need for words right now",
   "Cause I can feel you breathe\nIt's washing over me\nAnd suddenly I'm melting into you\nThere's nothing left to prove\nBaby all we need is just to be\nC

In [114]:
len(ALL_LYRICS)

1708

In [115]:
lyrics_df = pandas.DataFrame(ALL_LYRICS)

In [116]:
lyrics_df.to_csv('Scraped _LYRICS.csv')

{'Artist': 'Debelah Morgan',
 'Link': 'https://www.metrolyrics.com/debolah-morgan-dance-with-me-debelah-morgan-ml-video-ejp.html',
 'Song': 'Dance With Me'}